In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler

In [4]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%y%m%d%H%M%S")

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [6]:
config = {
    'batch_size': 32,
    'epoch': 50,
    'learning_rate': 1e-2,
    'seq_len': 26,
    'label_len': 2,
    'pred_len': 2,
    'individual': True,
    'enc_in': 0
}

dlinear_size = [config['seq_len'], config['label_len'], config['pred_len']]

In [7]:
# Fix seed
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

In [8]:
# 규격준수율 계산 식
def cal_accuracy_rate(actual, prediction):
    
    if prediction<0:
        prediction = 0

    result = 0
    if (prediction >= actual) and (prediction > 0):
        result = actual / prediction * 100
    elif (prediction < actual) and (prediction > 0):
        if actual > prediction * 2:
            result = 0
        else:
            result = (1 - (actual - prediction) / prediction) * 100
    elif prediction == 0 and actual > 0:
        result = 0
    elif prediction == 0 and actual == 0:
        result = -1
    else:
        print(actual, prediction)
        raise ValueError()
    return round(result, 1)

### 예측 대상 품목 추출

In [9]:
target_item = pd.read_excel("./data/2.주별_수요예측대상품목.xlsx")
target_item = target_item.iloc[np.where(target_item["bas_week"] == 2713)].reset_index(drop=True)
print(target_item.shape)
print(target_item["item_code"].nunique())
target_item.head()

D:\jupyter\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(3409, 2)
3409


,item_code,bas_week
0,1000029,2713.0
1,1000043,2713.0
2,1000138,2713.0
3,1000163,2713.0
4,1000184,2713.0


In [10]:
orig_data = pd.read_csv("./data/3.주문실적_2022_SW51.csv")
print("Data shape:", orig_data.shape)
print("Unique code num:", orig_data["자재코드"].nunique())
orig_data.head()

Data shape: (1986699, 5)
Unique code num: 11257


,자재코드,채널구분,주문주차,주문수량,결품수량
0,1000029,ETC,2015-SW01,198,1
1,1000029,NH,2015-SW01,12,0
2,1000043,CN,2015-SW01,365,0
3,1000043,ETC,2015-SW01,161,4
4,1000043,NH,2015-SW01,40,0


In [11]:
# 자재코드 및 주문주차별 합계로 y값 생성
data = orig_data.groupby(["자재코드", "주문주차"]).agg({"주문수량":"sum", "결품수량":"sum"}).reset_index()
data["y"] = data["주문수량"] - data["결품수량"]
data.head(10)

,자재코드,주문주차,주문수량,결품수량,y
0,1000004,2015-SW39,1,1,0
1,1000020,2015-SW02,12,0,12
2,1000020,2015-SW03,122,0,122
3,1000020,2015-SW04,114,0,114
4,1000020,2015-SW06,78,3,75
5,1000020,2015-SW07,2,0,2
6,1000020,2015-SW09,109,0,109
7,1000020,2015-SW11,124,12,112
8,1000020,2015-SW12,20,0,20
9,1000020,2015-SW13,32,0,32


In [12]:
data = data[data["자재코드"].isin(target_item["item_code"])].reset_index(drop=True)
print("추출 품목 데이터 shape:", data.shape)
print("대상 품목 개수:", data["자재코드"].nunique())
data.head()

추출 품목 데이터 shape: (687658, 5)
대상 품목 개수: 3408


,자재코드,주문주차,주문수량,결품수량,y
0,1000029,2015-SW01,210,1,209
1,1000029,2015-SW02,212,0,212
2,1000029,2015-SW03,186,0,186
3,1000029,2015-SW04,123,0,123
4,1000029,2015-SW05,257,0,257


In [13]:
# 학습 시간으로 인해 item_sal_cls가 A인 것들만 학습
item_master = pd.read_csv("./data/1.품목마스터.csv")
print(item_master.shape)

a_class_code = item_master.iloc[np.where(item_master["item_sal_cls"] == "A")]["item_cd"].unique()
data = data[data["자재코드"].isin(a_class_code)]
data["자재코드"].nunique()

(10832, 18)


858

### Calender feature merge

In [14]:
cal_data = pd.read_excel("./data/new_달력마스터.xlsx")
print(cal_data.shape)
cal_data.head()

(574, 24)


,bas_date,ord_week,bas_year,bas_week,bas_yweek,ymonth,mweek,bas_time,iy_time,im_time,...,n_nyd_tgd,tgd_ind,nyd_ind,chobok,joongbok,malbok,summer_vac,winter_vac,spring_vac,covid19
0,2014-12-29,2015-SW01,2015,2297,1,1,1,2014.994521,0.994521,0.935484,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-05,2015-SW02,2015,2298,2,1,2,2015.013699,0.013699,0.161290,...,0,0,0,0,0,0,0,1,0,0
2,2015-01-12,2015-SW03,2015,2299,3,1,3,2015.032877,0.032877,0.387097,...,0,0,0,0,0,0,0,1,0,0
3,2015-01-19,2015-SW04,2015,2300,4,1,4,2015.052055,0.052055,0.612903,...,0,0,-4,0,0,0,0,0,0,0
4,2015-01-26,2015-SW05,2015,2301,5,1,5,2015.071233,0.071233,0.838710,...,0,0,-3,0,0,0,0,0,0,0


In [15]:
data = pd.merge(data, cal_data, left_on="주문주차", right_on="ord_week", how="left")
data.head()

,자재코드,주문주차,주문수량,결품수량,y,bas_date,ord_week,bas_year,bas_week,bas_yweek,...,n_nyd_tgd,tgd_ind,nyd_ind,chobok,joongbok,malbok,summer_vac,winter_vac,spring_vac,covid19
0,1000043,2015-SW01,566,4,562,2014-12-29,2015-SW01,2015,2297,1,...,0,0,0,0,0,0,0,0,0,0
1,1000043,2015-SW02,512,2,510,2015-01-05,2015-SW02,2015,2298,2,...,0,0,0,0,0,0,0,1,0,0
2,1000043,2015-SW03,494,10,484,2015-01-12,2015-SW03,2015,2299,3,...,0,0,0,0,0,0,0,1,0,0
3,1000043,2015-SW04,480,3,477,2015-01-19,2015-SW04,2015,2300,4,...,0,0,-4,0,0,0,0,0,0,0
4,1000043,2015-SW05,493,4,489,2015-01-26,2015-SW05,2015,2301,5,...,0,0,-3,0,0,0,0,0,0,0


In [16]:
data.to_excel("./data/train_data_20230125.xlsx", index=False)

### DLinear 모델 정의

In [17]:
# 시계열 데이터 분해하는 부분
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)
        
    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        
        return x

class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)
    
    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        
        return res, moving_mean

In [18]:
# 1-layer linear network 구현 부분
class DLinear(nn.Module):
    """
    DLinear model
    """
    def __init__(self, configs):
        super(DLinear, self).__init__()
        self.seq_len = configs['seq_len']
        self.pred_len = configs['pred_len']
        
        # Decomposition Kernel Size
        kernel_size = 5
        self.decomposition = series_decomp(kernel_size)
        self.individual = configs['individual']
        self.channels = configs['enc_in']
        
        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            self.Linear_Decoder = nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len, self.pred_len))
                self.Linear_Seasonal[i].weight = nn.Parameter((1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))
                self.Linear_Trend.append(nn.Linear(self.seq_len, self.pred_len))
                self.Linear_Trend[i].weight = nn.Parameter((1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))
                self.Linear_Decoder.append(nn.Linear(self.seq_len, self.pred_len))
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len, self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len, self.pred_len)
            self.Linear_Decoder = nn.Linear(self.seq_len, self.pred_len)
            self.Linear_Seasonal.weight = nn.Parameter((1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))
            self.Linear_Trend.weight = nn.Parameter((1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len]))
    
    def forward(self, x):
        # x: [Batch, Input Length, Channel]
#         seasonal_init, trend_init = self.decomposition(x.unsqueeze(0).permute(1, 2, 0))
        seasonal_init, trend_init = self.decomposition(x)
        seasonal_init, trend_init = seasonal_init.permute(0, 2, 1), trend_init.permute(0, 2, 1)
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.pred_len], dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.pred_len], dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:, i, :] = self.Linear_Seasonal[i](seasonal_init[:, i, :])
                trend_output[:, i, :] = self.Linear_Trend[i](trend_init[:, i, :])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)
        
        x = seasonal_output + trend_output
        return x.permute(0, 2, 1)  # to [Batch, Output length, Channel]

### Dataset 정의

In [19]:
class Dataset_Custom(Dataset):
    def __init__(self, data=None, size=None, flag='train'):
        self.data = data
        
        # size = [seq_len, label_len, pred_len]
        self.seq_len = size[0]
        self.label_len = size[1]
        self.pred_len = size[2]
        self.flag = flag
        
        self.data_x = self.data[:len(self.data) - self.pred_len]
        self.data_y = self.data[:len(self.data) - self.pred_len]
    
    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len
        
        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        
        return seq_x, seq_y
    
    def __len__(self):
        if self.flag == 'train':
            return len(self.data_x) - self.seq_len - self.pred_len + 1
        else:
            return len(self.data_x) - self.seq_len + 1

### Train & inference 함수 생성

In [20]:
def train(model, optimizer, train_loader, device, epochs=1):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    
    for epoch in range(epochs, config['epoch'] + 1):
        model.train()
        train_loss = []
        
        for batch_x, batch_y in iter(train_loader):
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(batch_x)
            # univariate일 경우
#             output = output[:, -config['pred_len']:].flatten()
#             batch_y = batch_y[:, -config['pred_len']:].flatten()
            
            # multivariate일 경우
            output = output[:, -config['pred_len']:, -1:]
            batch_y = batch_y[:, -config['pred_len']:, -1:]
            loss = criterion(output, batch_y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())

        _train_loss = np.mean(train_loss)
#         print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}]')
    
    return model

In [21]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for batch_x, batch_y in iter(test_loader):
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            
            probs = model(batch_x)
            
            probs = probs.cpu().detach().numpy()
            if len(probs.shape) == 3:  # multivariate일 경우
                probs = probs[:, -config['pred_len']:, -1:]
            predictions.append(probs)

    predictions = predictions[0].flatten()
    return predictions

### BackTest

In [22]:
data.columns

Index(['자재코드', '주문주차', '주문수량', '결품수량', 'y', 'bas_date', 'ord_week', 'bas_year',
       'bas_week', 'bas_yweek', 'ymonth', 'mweek', 'bas_time', 'iy_time',
       'im_time', 'nyddist', 'tgddist', 'bhdist', 'n_holiday', 'n_nyd_tgd',
       'tgd_ind', 'nyd_ind', 'chobok', 'joongbok', 'malbok', 'summer_vac',
       'winter_vac', 'spring_vac', 'covid19'],
      dtype='object')

In [23]:
feat_col = ['bas_yweek', 'ymonth', 'mweek', 'nyddist', 'tgddist', 'bhdist', 'n_holiday', 'n_nyd_tgd', 'y']
config['enc_in'] = len(feat_col)

In [28]:
trg_weeks = [2709, 2710, 2711, 2712, 2713]

for trg_week in trg_weeks:
    result_df = pd.DataFrame()
    print(f"{trg_week}주차 학습 진행...")
    for code in tqdm(data["자재코드"].unique()):
        try:
            test_ts = data.iloc[np.where(data["자재코드"] == code)]
            test_ts = test_ts[test_ts["bas_week"] <= trg_week]
            train_cut = len(test_ts) - config['pred_len']
            test_cut = train_cut - config['seq_len']

            train_x = test_ts[feat_col].values[:train_cut]
            test_x = test_ts[feat_col].values[test_cut:]

            if np.average(test_ts.iloc[-config['pred_len'] - 4:-config['pred_len']]["y"]) == 0:  # 지난 4주동안의 평균이 0일 경우 0처리
                preds = 0
            else:
                # Train data loader 정의 및 학습
                train_dataset = Dataset_Custom(data=train_x, size=dlinear_size, flag='train')
                train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=0)

                model = DLinear(config)
                model.eval()
                optimizer = torch.optim.Adam(params=model.parameters(), lr=config["learning_rate"])

                trained_model = train(model, optimizer, train_loader, device)

                # Test data loader 정의 및 추론
                test_dataset = Dataset_Custom(data=test_x, size=dlinear_size, flag='pred')
                test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=0)

                preds = int(inference(model, test_loader, device)[-1])

            score = cal_accuracy_rate(test_x[-1, :][-1], preds)

            tmp_df = pd.DataFrame({
                "item_cd": [code],
                "bas_week": [trg_week],
                "DLinear_qty": [preds],
                "score": [score]
            })

            result_df = pd.concat([result_df, tmp_df], axis=0)
        except Exception as e:
            print(e)
            print("Error code:", code, "Length of data:", len(data.iloc[np.where(data["자재코드"] == code)]))
    
    result_df.to_excel(f"./result/a_class_DLinear_{trg_week}_{current_time}.xlsx", index=False)
    print(f"{trg_week} 학습 완료:", result_df.shape)
    print("-" * 35)

2709주차 학습 진행...


  0%|          | 0/858 [00:00<?, ?it/s]

__len__() should return >= 0
Error code: 1024146 Length of data: 27
__len__() should return >= 0
Error code: 1024241 Length of data: 31
__len__() should return >= 0
Error code: 1024243 Length of data: 34
__len__() should return >= 0
Error code: 1024318 Length of data: 33
__len__() should return >= 0
Error code: 1024319 Length of data: 24
__len__() should return >= 0
Error code: 1024615 Length of data: 28
__len__() should return >= 0
Error code: 1024622 Length of data: 29
__len__() should return >= 0
Error code: 1024629 Length of data: 32
__len__() should return >= 0
Error code: 1024646 Length of data: 28
__len__() should return >= 0
Error code: 1024680 Length of data: 26
__len__() should return >= 0
Error code: 1024816 Length of data: 19
__len__() should return >= 0
Error code: 1024838 Length of data: 19
__len__() should return >= 0
Error code: 2034599 Length of data: 23
__len__() should return >= 0
Error code: 2034758 Length of data: 13
__len__() should return >= 0
Error code: 2036111

  0%|          | 0/858 [00:00<?, ?it/s]

__len__() should return >= 0
Error code: 1024146 Length of data: 27
__len__() should return >= 0
Error code: 1024241 Length of data: 31


...

__len__() should return >= 0
Error code: 1024146 Length of data: 27


D:\jupyter\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\jupyter\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


__len__() should return >= 0
Error code: 1024319 Length of data: 24
__len__() should return >= 0
Error code: 1024615 Length of data: 28
__len__() should return >= 0
Error code: 1024622 Length of data: 29
__len__() should return >= 0
Error code: 1024646 Length of data: 28
__len__() should return >= 0
Error code: 1024680 Length of data: 26
__len__() should return >= 0
Error code: 1024816 Length of data: 19
__len__() should return >= 0
Error code: 1024838 Length of data: 19
__len__() should return >= 0
Error code: 2034599 Length of data: 23
__len__() should return >= 0
Error code: 2034758 Length of data: 13
__len__() should return >= 0
Error code: 2036111 Length of data: 21
__len__() should return >= 0
Error code: 2036469 Length of data: 21
__len__() should return >= 0
Error code: 2036470 Length of data: 28
__len__() should return >= 0
Error code: 2036939 Length of data: 26


D:\jupyter\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\jupyter\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\jupyter\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\jupyter\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


__len__() should return >= 0
Error code: 2037967 Length of data: 15
__len__() should return >= 0
Error code: 2037988 Length of data: 13
2713 학습 완료: (842, 4)
-----------------------------------


In [29]:
final_result_df = pd.DataFrame()

for trg_week in trg_weeks:
    tmp_result_df = pd.read_excel(f"./result/a_class_DLinear_{trg_week}_{current_time}.xlsx")
    final_result_df = pd.concat([final_result_df, tmp_result_df])

final_result_df.shape

(4189, 4)

In [30]:
final_result_df.to_excel(f"./result/a_class_DLinear_combine_result_{current_time}.xlsx", index=False)

### 기존 점수와 비교

In [31]:
pre_result = pd.read_csv("./data/4.수요예측결과_2021-SW28_2022-SW52.csv")
pre_result = pre_result.iloc[np.where(pre_result["bas_week"].isin(trg_weeks))]

compare_result = pd.merge(final_result_df, pre_result, how="left", on=["item_cd", "bas_week"])[["item_cd", "bas_week", "ord_qty", "fcst_qty", "sco_rate", "DLinear_qty", "score"]]
compare_result.to_excel(f"./result/a_class_DLinear_compare_result_{current_time}.xlsx", index=False)

In [32]:
print("기존 점수:", np.mean(compare_result[compare_result["sco_rate"] != -1]["sco_rate"]))
print("DLinear 점수:", np.mean(compare_result[compare_result["score"] != -1]["score"]))

기존 점수: 71.78890801770783
DLinear 점수: 65.52250479846448


In [33]:
for trg_week in trg_weeks:
    tmp_compare = compare_result[compare_result["bas_week"] == trg_week]
    print(f"{trg_week} 기존 점수:", np.mean(tmp_compare[tmp_compare["sco_rate"] != -1]["sco_rate"]))
    print(f"{trg_week} DLinear 점수:", np.mean(tmp_compare[tmp_compare["score"] != -1]["score"]))
    print("-" * 35)

print("기존 총평균 점수:", np.mean(compare_result[compare_result["sco_rate"] != -1]["sco_rate"]))
print("DLinear 총평균 점수:", np.mean(compare_result[compare_result["score"] != -1]["score"]))

2709 기존 점수: 70.8813349814586
2709 DLinear 점수: 66.24440433212997
-----------------------------------
2710 기존 점수: 72.96391625615763
2710 DLinear 점수: 66.78317307692306
-----------------------------------
2711 기존 점수: 72.85798525798526
2711 DLinear 점수: 66.03177458033574
-----------------------------------
2712 기존 점수: 71.69421894218942
2712 DLinear 점수: 64.85203836930457
-----------------------------------
2713 기존 점수: 70.55036674816625
2713 DLinear 점수: 63.71326164874552
-----------------------------------
기존 총평균 점수: 71.78890801770783
DLinear 총평균 점수: 65.52250479846448
